# Notebook for batch processing (Running Gears)
Niall Bourke  
11-11-24   

**Summary**
This script automates the process of searching for specific MRI files in a project, preparing them for analysis, and submitting them to run using a gear. It includes error handling and organizes files into the correct format for submission, making it an efficient way to manage MRI data processing.

This Python code is designed to loop through all the subjects and sessions in a project, and for each session, it will:
- Iterate through Acquisitions: The code iterates through all the acquisitions in each session, looking for specific Nifti files that match certain criteria (e.g., 'T2', 'AXI', 'COR', 'SAG', and excluding 'Segmentation' and 'Align'). These files are then stored in a dictionary called inputs.  
- Set Target Template: The code sets the target_template variable to 'None', which is used as the default configuration for the analysis.   
- Run Analysis: The code then attempts to run an analysis using the gear.run() function, passing in the analysis_label, inputs, destination (the current session), tags, and a configuration dictionary that includes the target_template and prefix values.  
- Error Handling: If there is an error running the analysis, the code will print a warning message.  

\* This example is running the MRR (Multi Resolution Reconstruction) gear, which takes AXI, COR, SAG acquisitions from Hyperfine Swoop as input to create an isotropic reconstruction   

**Notebook environment setup**
- Load required python packages
- Check connection to project through Flywheel SDK
- Set Gear options

---

### Setup options

In [1]:
import os
from datetime import datetime
import pytz
from IPython.display import display

In [2]:
display(f"User: {fw.get_current_user().id}")

project=fw_project
display(f"Project: {project.label}")

'User: nialljbourke@gmail.com'

'Project: ISMRM_UNITY_DEMO'

In [5]:
gear =  fw.lookup('gears/mrr')
analysis_tag = 'mrr-axireg'
 # Initialize gear_job_list
job_list = list()

### Main code block

In [7]:
# Loop through all the subjects in the project
for subject in project.subjects.iter():
    for session in subject.sessions.iter():
        session = session.reload()
        print("Parsing... ", subject.label, session.label)

        inputs = {}
        EXCLUDE_PATTERNS = ['Segmentation', 'Align', 'Mapping']
        INCLUDE_PATTERN = 'T2'
        PLANE_TYPES = ['AXI', 'COR', 'SAG']

        # Look at every acquisition in the session
        for acquisition in session.acquisitions.iter():
            acquisition = acquisition.reload()
            for file in acquisition.files:
                # We only want anatomical Nifti's
                if file.type == 'nifti' and INCLUDE_PATTERN in file.name:
                    if all(pattern not in file.name for pattern in EXCLUDE_PATTERNS):
                        for plane in PLANE_TYPES:
                            if plane in file.name:
                                input_label = plane.lower()
                                inputs[input_label] = file
                                print("inputs: ", file.name)
                                break

        # Set to 'None' in order to run axi registration by default
        target_template = 'None'

        try:
            # The destination for this analysis will be on the session
            dest = session
            time_fmt = '%d-%m-%Y_%H-%M-%S'
            analysis_label = f'{analysis_tag}_{datetime.now().strftime(time_fmt)}'
            job_id = gear.run(
                analysis_label=analysis_label,
                inputs=inputs,
                destination=dest,
                tags=[''],
                config={
                    "target_template": target_template,
                    "prefix": analysis_tag,
                }
            )
            job_list.append(job_id)
            print("Submitting Job: Check Jobs Log", dest.label)
        except Exception as e:
            print(f"WARNING: Job cannot be sent for {dest.label}. Error: {e}")


Parsing...  DEMO-001 2023-06-09 13_54_24
inputs:  4_T2_AXI_Fast.nii.gz
inputs:  6_T2_SAG_Fast.nii.gz
inputs:  5_T2_COR_Fast.nii.gz
Submitting Job: Check Jobs Log 2023-06-09 13_54_24


---

# Code Walkthrough

### 1. Loop Through All Subjects and Sessions

**Loop through all the subjects in the project**

```
for subject in project.subjects.iter():  
    for session in subject.sessions.iter():  
        session = session.reload()  
        print("parsing... ", subject.label, session.label)  

```

- **project.subjects.iter():** This iterates over each subject in the project.  
- **subject.sessions.iter():** This iterates over each session for the current subject.  
- **session.reload():** Reloads the session data to ensure it is up-to-date.  
- **print("parsing... ", subject.label, session.label):** Prints the subject and session labels to show which session is currently being processed.  

### 2. Prepare to Identify Files

`inputs = {}`

- **inputs:* An empty dictionary created to store specific MRI data files that meet our criteria.
- Resetting this distionary to empty is important to make sure a previous subjects files does not exist in it, if the current subject has no matching acquisitions



### 3. Loop Through Acquisitions in the Session

```
# Look at every acquisition in the session
for acq in session.acquisitions.iter():
    acq = acq.reload()
    for file_obj in acq.files:
        # We only want anatomical Nifti's
```

**session.acquisitions.iter():** Loops over all the acquisitions (collections of related images) within the session.  
**acq.reload():** Reloads acquisition data to ensure it is current.  
**file_obj:** Represents each file in the acquisition. The script checks if these files are anatomical MRI images in the NIfTI format.  


### 4. Identify Specific NIfTI Files
The script searches for files matching certain criteria:

``` 
if file_obj.type == 'nifti' and 'T2' in file_obj.name and 'AXI' in file_obj.name and 'Segmentation' not in file_obj.name and 'Align' not in file_obj.name:
    input_label = 'axi'
    inputs[input_label] = file_obj
    print("inputs: ", file_obj.name)
```

**file_obj.type == 'nifti':** Ensures the file is of type 'nifti', a common format for storing MRI data.  
**'T2' in file_obj.name:** Checks if the file name contains 'T2', indicating a T2-weighted MRI.  
**'AXI' in file_obj.name:** Searches for axial plane images ('AXI' indicates axial).  
**Segmentation and Align Checks:** Ensures the file name does not contain 'Segmentation' or 'Align', filtering out irrelevant files.  
**inputs[input_label] = file_obj:** Stores the matching file in the inputs dictionary, using 'axi', 'cor', or 'sag' as keys.  

The same logic applies to identifying:  
**'COR':** Coronal plane images.  
**'SAG':** Sagittal plane images.  


### 5. Run the Analysis

```
try:
    dest = session
    time_fmt = '%d-%m-%Y_%H-%M-%S'
    analysis_label = f'mrr_axireg_{datetime.now().strftime(time_fmt)}'
    job_id = gear.run(analysis_label=analysis_label, inputs=inputs, destination=dest, tags=['BIDS'], config={
        "target_template": target_template,
        "prefix": "mrr-axireg",
    })
    job_list.append(job_id)
    print("Submitting Job: Check Jobs Log", dest.label)
except:
    print("WARNING: Job cannot be sent.. ", dest.label)
    
```

**try-except Block:** Attempts to submit a job for analysis and handles errors gracefully.  
**dest = session:** Sets the session as the destination where the analysis results will be stored.  
**time_fmt:** Defines a date-time format for naming the analysis.  
**analysis_label:** Creates a unique label for the analysis using the current date and time.   
**gear.run(...):** Submits the job for analysis with:  
- **analysis_label:** The label for this analysis run.  
- **inputs:** The dictionary of identified files.  
- **destination:** The session where the results should be saved.  
- **tags:** Adds metadata tags, such as 'BIDS', for organization.  
- **config:** Includes parameters like target_template and a prefix for the analysis.  
- **job_list.append(job_id):** Adds the job ID to a list to track all submitted jobs.  

**print("Submitting Job: Check Jobs Log", dest.label):** Prints confirmation that the job was submitted.  
**except Block:** If an error occurs, prints a warning message indicating the job couldn’t be sent.  